In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.callbacks import Callback,EarlyStopping


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# load saved model
import tensorflow
tensorflow.keras.backend.set_floatx('float16')
ResNet50_model = tensorflow.keras.models.load_model('/content/drive/MyDrive/Project/Code/ipynb_checkpoints/ResNet50_model.h5')

In [4]:
# load the augmented test data
import torch
test_images, labels = torch.load('/content/drive/MyDrive/Project/Code/data/test_sets/small_1000_set.pth')

In [5]:
test_images = test_images.numpy()

test_images.shape

(1000, 3, 227, 227)

In [6]:
images = []
for test_img in test_images:
    images.append(np.moveaxis(test_img, 0, -1))
    
images = np.array(images)

images.shape

(1000, 227, 227, 3)

In [ ]:
from PIL import Image 

for i in range(1000):
    
    path = '/content/drive/MyDrive/Project/Code/data/test_sets/'
    if labels[i] == 0:
        path += '0/'
    else:
        path += '1/'
    
    img_arr = np.squeeze(images[i])
    _array = img_arr * 255
    _array = _array.astype(np.uint8)
    _image = Image.fromarray(_array)
    _image = _image.save(path + str(i) + '.jpg')

In [7]:
path = '/content/drive/MyDrive/Project/Code/data/test_sets/'
path_all = list(glob.glob(path+'**/*jpg'))
path_Negative = path + '0/*.jpg'
path_Positive = path + '1/*.jpg'

In [8]:
labels = list(map(lambda x:os.path.split(os.path.split(x)[0])[1], path_all))
file_Path = pd.Series(path_all, name='File_Path').astype(str)
labels = pd.Series(labels, name='Label')
data = pd.concat([file_Path, labels], axis=1)
data = data.sample(frac=1).reset_index(drop=True)

In [9]:
data

,File_Path,Label
0,/content/drive/MyDrive/Project/Code/data/test_...,1
1,/content/drive/MyDrive/Project/Code/data/test_...,0
2,/content/drive/MyDrive/Project/Code/data/test_...,0
3,/content/drive/MyDrive/Project/Code/data/test_...,1
4,/content/drive/MyDrive/Project/Code/data/test_...,1
...,...,...
995,/content/drive/MyDrive/Project/Code/data/test_...,1
996,/content/drive/MyDrive/Project/Code/data/test_...,0
997,/content/drive/MyDrive/Project/Code/data/test_...,1
998,/content/drive/MyDrive/Project/Code/data/test_...,1


In [10]:
from tensorflow.keras.applications.resnet50  import preprocess_input

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# data.head()
test_gen = test_datagen.flow_from_dataframe(
    dataframe=data,
    x_col='File_Path',
    y_col='Label',
    target_size=(100,100),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False)

Found 1000 validated image filenames belonging to 2 classes.


In [11]:
loss,accuracy = ResNet50_model.evaluate(test_gen, verbose = 0)

In [12]:
print(f'The loss of this ResNet50 is {loss:.3f}, and the accuracy is {(accuracy*100):.3f}%')

The loss of this ResNet50 is 0.301, and the accuracy is 96.100%
